**EirGrid-Scraping**

### Co2 Forecast 

This Python script is designed to fetch CO2 data from a specific API. It includes the following key components:

1. **Round Down Time Function (`round_down_time`)**
   - Rounds down the current time to the nearest half-hour, accommodating the data's 30-minute availability intervals.

2. **URL Creation Function (`create_url`)**
   - Constructs the API URL using formatted start and end times and a chosen region.

3. **Time and Region Settings**
   - Sets the current time (`now`), applies the round-down function, and calculates the end time as 24 hours from the current time.
   - Selects a region (example: 'ROI') for data fetching.

4. **API Request**
   - Generates the API URL and performs a GET request to retrieve the data.
   - Stores the response in JSON format.

This script is tailored for accessing time-sensitive environmental data from a designated API endpoint.


In [66]:
from datetime import datetime, timedelta
import requests
from pandas import DataFrame

# data is availble every 30 minutes, so we need to start at the nearest half-hour 
def round_down_time(dt):
    # Round down to the nearest half-ho/ur
    new_minute = 30 if dt.minute >= 30 else 0
    return dt.replace(minute=new_minute, second=0, microsecond=0)

# Function to create the API URL based on the start and end datetimes and the region
def create_url(start_datetime, end_datetime, region):
    start_str = start_datetime.strftime("%Y%m%d%H%M")
    end_str = end_datetime.strftime("%Y%m%d%H%M")
    url = f"https://www.co2.smartgriddashboard.com/api/co2_fc/{start_str}/{end_str}/{region}"
    return url

# Current date and time
now = datetime.now()
# Round down to the nearest half-hour
start_time = round_down_time(now)
# For end time, let's use 24 hours from now
end_time = now.replace(hour=23, minute=59, second=59, microsecond=0) #now + timedelta(days=1)

# Define the region
region = ['ROI', 'NI', 'ALL']

# Create the URL
api_url = create_url(start_time, end_time, region[2])
response = requests.get(api_url)
Rs = response.json()


In [67]:
import pandas as pd
df_carbon_forecast = pd.DataFrame(Rs['Rows'])

# Convert 'EffectiveTime' to datetime and set as index
df_carbon_forecast['EffectiveTime'] = pd.to_datetime(df_carbon_forecast['EffectiveTime'], format='%d-%b-%Y %H:%M:%S')
df_carbon_forecast_indexed = df_carbon_forecast.set_index('EffectiveTime')

# Now you can slice the DataFrame using datetime index
# # For example, to access data at '26-JAN-2024 21:30:00'
# specific_time_data = df_carbon_forecast_indexed.loc['2024-01-26 21:30:00']

# # To access a range of times
# time_range_data = df_carbon_forecast_indexed['2024-01-26 21:00:00':'2024-01-26 22:00:00']

# # Print to see the results
# print(specific_time_data)
# print(time_range_data)

In [68]:
df_carbon_forecast_indexed

,FieldName,Region,Value
EffectiveTime,,,
2024-01-26 23:00:00,CO2_INTENSITY_FORECAST,ALL,147.9230
2024-01-26 23:30:00,CO2_INTENSITY_FORECAST,ALL,148.8191


### Co2 intensity (measured) from day before

In [27]:
import datetime

# Function to round time to the nearest 15 minutes
def round_time(dt):
    # Round minutes to the nearest 15
    new_minute = (dt.minute // 15) * 15
    return dt.replace(minute=new_minute, second=0, microsecond=0)

# Function to format date in your specific format
def format_date(dt):
    return dt.strftime("%d-%b-%Y").lower() + "+" + dt.strftime("%H%%3A%M")

# Current date and time, rounded to the nearest 15 minutes
now = round_time(datetime.datetime.now())

# Start time (same time yesterday, rounded to the nearest 15 minutes)
yesterday = now - datetime.timedelta(days=1)
startDateTime = format_date(yesterday)

# End time (current time, rounded to the nearest 15 minutes)
endDateTime = format_date(now)

# Print formatted start and end times
print("Start Time:", startDateTime)
print("End Time:", endDateTime)


Start Time: 25-jan-2024+22%3A30
End Time: 26-jan-2024+22%3A30


In [69]:
import requests , json

area = ['CO2Stats', 'generationactual', 'co2emission', 'co2intensity', 'interconnection', 'SnspAll', 'frequency', 'demandactual', 'windactual']
region = ['ROI', 'NI', 'ALL']
Rows = []

url = f'http://smartgriddashboard.eirgrid.com/DashboardService.svc/data?area={area[3]}&region={region[2]}&datefrom={startDateTime}&dateto={endDateTime}'
response = requests.get(url)
Rs = json.loads(response.text)["Rows"]
for row in Rs:
    Rows.append(row)

df_carbon_intensity_day_before= DataFrame(Rows)

In [70]:
# Convert 'EffectiveTime' to datetime and set as index
df_carbon_intensity_day_before['EffectiveTime'] = pd.to_datetime(df_carbon_intensity_day_before['EffectiveTime'], format='%d-%b-%Y %H:%M:%S')
df_carbon_intensity_indexed = df_carbon_intensity_day_before.set_index('EffectiveTime')

In [71]:
import pandas as pd
import numpy as np


# Interpolate NaN values, except for the last NaN
df_carbon_int_interp = df_carbon_intensity_indexed.interpolate()

# Reverse the DataFrame
df_reversed = df_carbon_int_interp.iloc[::-1]

# Find the first valid index in the reversed DataFrame, which corresponds to the last NaN before non-NaN values
last_nan_index_before_non_nan = df_reversed['Value'].first_valid_index()


# Select rows up to the row before the last NaN
df_carbon_intensity_recent = df_carbon_int_interp.loc[:last_nan_index_before_non_nan]

/var/folders/n8/5rf_2zc91lx1ffhm5t27hrsw0000gn/T/ipykernel_3519/2998555976.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_carbon_int_interp = df_carbon_intensity_indexed.interpolate()


In [72]:
last_nan_index_before_non_nan

Timestamp('2024-01-26 22:30:00')

In [79]:
import json

# Calculate mean, min, and max
mean_val = df_carbon_intensity_recent['Value'].mean()
min_val = df_carbon_intensity_recent['Value'].min()
max_val = df_carbon_intensity_recent['Value'].max()

# Create a dictionary with these values
co2_stats_prior_day = {
    "mean": mean_val,
    "min": min_val,
    "max": max_val
}

# Convert the dictionary to a JSON string
co2_stats_json_prior_day = json.dumps(co2_stats_prior_day)

print(co2_stats_json_prior_day)


{"mean": 143.75257731958763, "min": 126, "max": 161}


In [83]:
co2_stats_prior_day['max']

161

In [87]:
df_carbon_forecast_indexed

,FieldName,Region,Value
EffectiveTime,,,
2024-01-26 23:00:00,CO2_INTENSITY_FORECAST,ALL,147.9230
2024-01-26 23:30:00,CO2_INTENSITY_FORECAST,ALL,148.8191


In [88]:
df_carbon_forecast_value = df_carbon_forecast_indexed['Value']

carbon_forecast_hourly = df_carbon_forecast_value.resample('1H').mean()

In [93]:
pd.DataFrame(carbon_forecast_hourly)

,Value
EffectiveTime,
2024-01-26 23:00:00,148.37105
